# Acesso a bases de dados MySQL com Python  
Autor do Documento: Prof. Pedro Cardoso
(ISE/UAlg - pcardoso@ualg.pt)

Adaptações e Docente que leciona: Prof. Cristiano Cabrita
(ISE/UAlg - ccabrita@ualg.pt)
---

## **Objetivos**: 

- Compreender o conceito de programação cliente-servidor
e aplicação desktop. 
- Aprender a aceder a bases de dados em
Python. 
- Aprender os fundamentos para construção de aplicações
em Python capazes de consultar dados de bases de dados.
- Aprender a construir aplicações em Python capazes de inserir,
alterar e excluir dados de bases de dados.
---

## Estabelecimento de conexão à base de dados usando um Connector/Python

o método  `connect()` cria uma conexão a um servidor MySQL e devolve um objeto `MySQLConnection`.
(ver https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html para outros argumentos e opções)

In [1]:
import mysql.connector

cnx = mysql.connector.connect(user='sensors', 
                              password='##sensors##',
                              host='localhost', # replace 'localhost', if necessary
                              database='sensors')
cnx

Algumas informações sobre a conexão podem ser consultadas no `__dict__`

In [2]:
dir(mysql.connector)

['BINARY',
 'Binary',
 'CharacterSet',
 'ClientFlag',
 'Connect',
 'DATETIME',
 'DEFAULT_CONFIGURATION',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'ERROR_NO_CEXT',
 'Error',
 'FieldFlag',
 'FieldType',
 'HAVE_CEXT',
 'HAVE_DNSPYTHON',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'MySQLConnection',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'PoolError',
 'ProgrammingError',
 'ROWID',
 'RefreshOption',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '_CONNECTION_POOLS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '__version_info__',
 '_get_failover_connection',
 '_get_pooled_connection',
 'abstracts',
 'apilevel',
 'authentication',
 'charsets',
 'connect',
 'connection',
 'constants',
 'conversion',
 'cursor',
 'custom_error_exception',
 'custom_types',
 'dbapi',
 'errorcode',
 'errors',
 'locale

E no final devemos libertar sempre a conexão

In [3]:
cnx.close()

### Ficheiro de configuração  e tratamento de exceções
De um modo geral é aconselhável
* fazer tratamento de exceções e,
* criar um ficheiro de configuração (config.py) 
```
config = {
    'host' : 'localhost',
    'user' : 'sensors',
    'password' : '##sensors##',
    'db' : 'sensors'
}
```
e depois fazer...

In [6]:
# Comecamos por importar o ficheiro de configuração
import mysql.connector

config = {
  'host' : 'localhost',
  'user' : 'sensors',
  'password' : '##sensors##',
  'db' : 'sensors'
}

In [7]:
try:
    cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
    print('Ups! Ocorreu um erro!')
    print(dir(err))
    print(err.errno)
else:
    print('Sucesso!')
    cnx.close()

Sucesso!


### Exercício
1. Experimente desligar o servidor e correr a linha acima: qual a mensagem de erro?
2. Mude o nome do utilizador no ficheiro de configuração (reinicie o kernel) e corra a linha acima: qual a mensagem de erro?
3. Re-implemente o código de modo a dar a mensagem adequada quando falhar devido ao servidor estar desligado e voltar a tentar mais 2 vezes a cada 5 segundos (vejam o pacote `time` e particular o método `sleep()`)

In [8]:
import time
print('ola')
time.sleep(5)
print('ola de novo!')

ola
ola de novo!


In [11]:
# Comecamos por importar o ficheiro de configuração
import mysql.connector
import time


config = {
  'host' : 'localhost',
  'user' : 'sensors',
  'password' : '##sensors##',
  'db' : 'sensors'
}

while True:
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        print('Ups! Ocorreu um erro!')
        print(dir(err))
        print(err.errno)
    else:
        print('Sucesso!')
        cnx.close()
        break
    time.sleep(5)
    

Sucesso!


## Operações de DDL: Criação de uma base de dados

Para a criação das tabelas e relacionamentos podemos construir o sql ou, como alternativa, podemos usar ferramentas como sejam o MySQL Workbench, o Phpmyadmin, o SQlite Browser, o DataGrip, etc. 


Consideremos o caso em que contruímos o sql...

Comecemos por criar uma base de dados no servidor de MySQL (façam sempre tratamento de exceções...).


Abaixo, os campos que pretendemos criar e respetivas tabelas:

In [12]:
sql = '''
    -- -----------------------------------------------------
    -- Table `sensors`.`Location`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Location` (
      `idLocation` INT NOT NULL AUTO_INCREMENT,
      `name` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idLocation`),
      UNIQUE INDEX `name_UNIQUE` (`name` ASC)) ENGINE = InnoDB);


    -- -----------------------------------------------------
    -- Table `sensors`.`Unit`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Unit` (
      `unit` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`unit`)) ENGINE = InnoDB);


    -- -----------------------------------------------------
    -- Table `sensors`.`Sensor`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Sensor` (
      `idSensor` INT NOT NULL AUTO_INCREMENT,
      `idLocation` INT NOT NULL,
      `name` VARCHAR(45) NOT NULL,
      `unit` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idSensor`),
      INDEX `fk_Sensor_Location_idx` (`idLocation` ASC),
      INDEX `fk_Sensor_Units1_idx` (`unit` ASC),
      UNIQUE INDEX `uniq_loc_vs_sensor` (`idLocation` ASC, `name` ASC),
      CONSTRAINT `fk_Sensor_Location`
        FOREIGN KEY (`idLocation`)
        REFERENCES `sensors`.`Location` (`idLocation`)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
      CONSTRAINT `fk_Sensor_Units1`
        FOREIGN KEY (`unit`)
        REFERENCES `sensors`.`Unit` (`unit`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB);


    -- -----------------------------------------------------
    -- Table `sensors`.`Reading`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Reading` (
      `idReading` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `value` FLOAT NOT NULL,
      PRIMARY KEY (`idReading`),
      INDEX `fk_Reading_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Reading_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `sensors`.`Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB);


    -- -----------------------------------------------------
    -- Table `sensors`.`Alert`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Alert` (
      `idAlert` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      `cleared` BIT NULL,
      PRIMARY KEY (`idAlert`),
      INDEX `fk_Alert_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Alert_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `sensors`.`Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB);
'''

Depois de termos o SQL/DDL, o código seguinte procede de acordo:
- testa a existência da base de dados, caso contrário usa a função *create_database(cursor)*
- cria um dicionário com a informação das tabelas, de acordo com a notação DDL
- cria as tabelas a partir do dicionário (função *load_tables*)
- o tratamento de exceções é recomendado, para mais info sobre os códigos: https://mariadb.com/kb/en/mariadb-error-codes/
No entanto, e para ilustrar na primeira tabela (*location*) é usada a notação ``` IF NOT EXISTS ```

In [14]:
# from config import config as conf
#from config import config #recorre às configurações definidas no ficheiro config.py
from mysql.connector import errorcode # acede aos diferentes erros de acesso à base de dados mariadb (former mysql)
config = {
  'host' : 'localhost',
  'user' : 'sensors',
  'password' : '##sensors##',
  'db' : 'sensors'
}




TABLES = {}
TABLES['location'] = ('''
    CREATE TABLE IF NOT EXISTS `location` (
      `idLocation` INT NOT NULL AUTO_INCREMENT,
      `name` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idLocation`),
      UNIQUE INDEX `name_UNIQUE` (`name` ASC)) ENGINE = InnoDB
      ''');

TABLES['unit'] = ('''
    CREATE TABLE `unit` (
      `unit` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`unit`)) ENGINE = InnoDB'''
                  );

TABLES['sensor'] = ('''
     CREATE TABLE `sensor` (
      `idSensor` INT NOT NULL AUTO_INCREMENT,
      `idLocation` INT NOT NULL,
      `name` VARCHAR(45) NOT NULL,
      `unit` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idSensor`),
      INDEX `fk_Sensor_Location_idx` (`idLocation` ASC),
      INDEX `fk_Sensor_Units1_idx` (`unit` ASC),
      UNIQUE INDEX `uniq_loc_vs_sensor` (`idLocation` ASC, `name` ASC),
      CONSTRAINT `fk_Sensor_Location`
        FOREIGN KEY (`idLocation`)
        REFERENCES `Location` (`idLocation`)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
      CONSTRAINT `fk_Sensor_Units1`
        FOREIGN KEY (`unit`)
        REFERENCES `Unit` (`unit`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB'''
        );

                                  
TABLES['Reading'] = ('''
 CREATE TABLE `Reading` (
      `idReading` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `value` FLOAT NOT NULL,
      PRIMARY KEY (`idReading`),
      INDEX `fk_Reading_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Reading_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB'''
      );


                  
                  
TABLES['Alert'] = ('''                  
 CREATE TABLE `Alert` (
      `idAlert` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      `cleared` BIT NULL,
      PRIMARY KEY (`idAlert`),
      INDEX `fk_Alert_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Alert_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB'''
        );

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

def load_tables(cursor, TABLES):
    for table_name in TABLES:
        table_description = TABLES[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("Table already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

        
try:
    DB_NAME = 'sensors'
    cnx = mysql.connector.connect(**config)
    # criamos um cursor a partir da conexão 
    cursor = cnx.cursor()
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(DB_NAME))
    if err.errno == 1050:
    #errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)
else:
    load_tables(cursor, TABLES)

Creating table location: OK
Creating table unit: Table already exists.
Creating table sensor: Table already exists.
Creating table Reading: Table already exists.
Creating table Alert: Table already exists.


O comando `cnx.cursor()` devolve um objeto da classe `MySQLCursor` que podem executar operações como instruções SQL. Objetos de `cursor` interagem com o servidor MySQL usando um objeto `MySQLConnection`. Para mais informações ver https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor.html

No phpmyadmin devem ver algo como

![phpmyadmin_database_created.png](phpmyadmin_database_created.png)


## Operações CRUD
---
### Ver manualweschools sobre syntax das operações CRUD: https://www.w3schools.com/sql/default.asp


### `INSERT` 

Abra a conexão em MySQL

In [15]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()


#### Localização
Uma boa estratégia é definir variáveis no SQL usando parametros no estilo `%s` or `%(nome)` (i.e., usar o estilo "format" ou "pyformat" - ver https://pyformat.info/) e um tuplo com os dados

In [17]:
sql = '''
    INSERT INTO Location 
        (idLocation, name, description) 
    VALUES 
        (DEFAULT, %s, %s)
'''
# e um tuplo com os dados
data = ('Ualg Server', 'Ualg Server @ lab. 167 / ISE /UAlg')

e agora inserir uma nova localização na base de dados e obter o id correspondente, guardado em `location_id` e que iremos usar à frente

In [18]:
cursor.execute(sql, data)
location_id = cursor.lastrowid
location_id

1

Importante, quando estamos a usar um sistema transacional, como o InnoDB, temos de efetuar o "commit" depois de fazer um INSERT, DELETE, ou UPDATE (comandos que alterem tabelas).
ver (https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlconnection-commit.html)

In [19]:
cnx.commit()

## vamos substituir ID de uma entrada da tabela Location

In [20]:
sql = '''
    UPDATE Location
       SET idLocation=3, name=%s, description=%s
    WHERE idLocation=2
'''
# e um tuplo com os dados
data = ('Ualg Server', 'Ualg Server @ lab. 167 / ISE /UAlg')
cursor.execute(sql, data)
cnx.commit()

#### Unit
Inserir uma nova `Unit`, se não existir (ver a documentação do comando `REPLACE` do SQL)

In [21]:
sql = '''
    REPLACE INTO Unit 
        (unit, description) 
    VALUES 
        (%s, %s)
'''
data = ("percent", "percentage of usage")

cursor.execute(sql, data)
cnx.commit()

#### Sensor
Inserir um novo sensor e obter o seu id, preparando o sql

In [22]:
# prepare o sql
sql = '''INSERT INTO Sensor 
            (idSensor, idLocation, name, unit)
        VALUES 
            (DEFAULT, %(idLocation)s, %(name)s, %(unit)s);'''

Preparar os dados, agora com um dicionário que tem em conta `%(idLocation)s, %(name)s, %(unit)s`


In [23]:
data = {
        'idLocation': location_id, 
        'name' : 'cpu_sensor_01', 
        'unit' : 'percent'
       }

Executar o sql query

In [24]:
cursor.execute(sql, data)
sensor_id = cursor.lastrowid
cnx.commit()

In [25]:
sensor_id

1

#### Readings
E agora, obter alguns dados e enviar para a base de dados. Neste caso vamos usar a biblioteca `psutil` que permite obter informações sobre processos em execução e utilização do sistema (CPU, memória, discos, rede, sensores) em Python. (https://pypi.org/project/psutil/)

In [51]:
!pip3 install psutil
import psutil
psutil.cpu_percent(interval=1)

  Using cached psutil-5.8.0.tar.gz (470 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for psutil, since package 'wheel' is not installed.
    Running setup.py install for psutil: started
    Running setup.py install for psutil: finished with status 'error'


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\marcramos\AppData\Local\Programs\Python\Python310\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\marcramos\\AppData\\Local\\Temp\\pip-install-7v1ndn0o\\psutil_81c766f02b89448eb6fd953f777be337\\setup.py'"'"'; __file__='"'"'C:\\Users\\marcramos\\AppData\\Local\\Temp\\pip-install-7v1ndn0o\\psutil_81c766f02b89448eb6fd953f777be337\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\marcramos\AppData\Local\Temp\pip-record-kbq6x7ps\install-record.txt' --single-version-externally-managed --compile --install-headers 'C:\Users\marcramos\AppData\Local\Programs\Python\Python310\Include\psutil'
         cwd: C:\Users\marcramos\AppData\Lo

ModuleNotFoundError: No module named 'psutil'

In [ ]:
import psutil

sql = '''
INSERT INTO Reading
    (idReading, idSensor, timestamp, value)     
VALUES 
    (DEFAULT, %(idSensor)s, DEFAULT, %(value)s)
'''

for _ in range(20):
    data = {
           'idSensor' : sensor_id, 
           'value' : psutil.cpu_percent(interval=1)
           }
    
    cursor.execute(sql, data) 
    cnx.commit()
    print('.', end='')

In [ ]:
cursor.close()
cnx.close()

---

# Selecionar dados em MySQL

Todo o processo é simples dados os conhecimentos anteriores</div><i class="fa fa-lightbulb-o "></i>

In [52]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()  

In [53]:
sql = '''
    SELECT idLocation, name, description 
    FROM Location 
    WHERE description LIKE "%163%"
'''

cursor.execute(sql)

In [54]:
cursor

E percorrer os dados, usando por exemplo um ciclo `for`

In [55]:
for (idLocation, name, description) in cursor:
    print("id: {}\n\t name: {} \n\t description: {}".format(idLocation, name, description))


Outros exemplos

In [56]:
sql = '''
    SELECT idReading, idSensor, timestamp, value 
    FROM Reading 
    WHERE value BETWEEN %s and %s
'''
data = (20, 30)

cursor.execute(sql, data)

for (idReading, idSensor, timestamp, value) in cursor:
    print("idReading: {}\n\t idSensor: {} \n\t time: {} \n\t value: {}".format(idReading, idSensor, timestamp, value))

In [57]:
sql = '''
    select *
    from Location
        inner join Sensor S on Location.idLocation = S.idLocation
        inner join Unit U on S.unit = U.unit
        inner join Reading R on S.idSensor = R.idSensor
    where value between %(low)s and %(high)s
    order by value
'''

data = {
    'low': 20,
    'high': 30
}

cursor.execute(sql, data)
#list(cursor)

Podemos obter os nomes e outros dados das colunas (https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-description.html) no formato
(name, type_code, display_size, internal_size, precision, scale, null_ok, column_flags)


In [58]:
cursor.description

[('idLocation', 3, None, None, None, None, 0, 1, 63),
 ('name', 253, None, None, None, None, 0, 4097, 45),
 ('description', 253, None, None, None, None, 0, 4097, 45),
 ('idSensor', 3, None, None, None, None, 0, 1, 63),
 ('idLocation', 3, None, None, None, None, 0, 4097, 63),
 ('name', 253, None, None, None, None, 0, 4097, 45),
 ('unit', 253, None, None, None, None, 0, 4097, 45),
 ('unit', 253, None, None, None, None, 0, 4097, 45),
 ('description', 253, None, None, None, None, 0, 4097, 45),
 ('idReading', 3, None, None, None, None, 0, 1, 63),
 ('idSensor', 3, None, None, None, None, 0, 4097, 63),
 ('timestamp', 7, None, None, None, None, 0, 161, 63),
 ('value', 4, None, None, None, None, 0, 4097, 63)]

In [59]:
from mysql.connector import FieldType

for i in range(len(cursor.description)):
    print("Column {}:".format(i+1))
    desc = cursor.description[i]
    print("  column_name = {}".format(desc[0]))
    print("  type = {} ({})".format(desc[1], FieldType.get_info(desc[1])))
    print("  null_ok = {}".format(desc[6]))

Column 1:
  column_name = idLocation
  type = 3 (LONG)
  null_ok = 0
Column 2:
  column_name = name
  type = 253 (VAR_STRING)
  null_ok = 0
Column 3:
  column_name = description
  type = 253 (VAR_STRING)
  null_ok = 0
Column 4:
  column_name = idSensor
  type = 3 (LONG)
  null_ok = 0
Column 5:
  column_name = idLocation
  type = 3 (LONG)
  null_ok = 0
Column 6:
  column_name = name
  type = 253 (VAR_STRING)
  null_ok = 0
Column 7:
  column_name = unit
  type = 253 (VAR_STRING)
  null_ok = 0
Column 8:
  column_name = unit
  type = 253 (VAR_STRING)
  null_ok = 0
Column 9:
  column_name = description
  type = 253 (VAR_STRING)
  null_ok = 0
Column 10:
  column_name = idReading
  type = 3 (LONG)
  null_ok = 0
Column 11:
  column_name = idSensor
  type = 3 (LONG)
  null_ok = 0
Column 12:
  column_name = timestamp
  type = 7 (TIMESTAMP)
  null_ok = 0
Column 13:
  column_name = value
  type = 4 (FLOAT)
  null_ok = 0


In [60]:
lista_de_colunas = [linha[0] for linha in cursor.description]
lista_de_colunas

['idLocation',
 'name',
 'description',
 'idSensor',
 'idLocation',
 'name',
 'unit',
 'unit',
 'description',
 'idReading',
 'idSensor',
 'timestamp',
 'value']

In [61]:
for linha in cursor:
    print('\t'.join([f'|{coluna}: {valor}' for coluna, valor in zip(lista_de_colunas, linha)]))    

### o comando `fetchall`
Usando o comando `fetchall` podemos obter todos os resultados de uma única vez como uma lista de tuplos

In [62]:
# é necessario voltar a correr o select pois o cursor foi esvaziado
cursor.execute(sql, data)

cursor.fetchall()

[]

Podemos também converter para um dicionário mas __nosso exemplo (dado o "*") NÃO é boa ideia__ pois há colunas que "têm o mesmo nome" (e.g., `nome`), pelo que se perdem colunas ao passar para um dicionário.

In [63]:
# é necessario voltar a correr o select pois o cursor foi "esvaziado"
cursor.execute(sql, data)

for linha in cursor:
    print({coluna: valor for valor, coluna  in zip(linha, lista_de_colunas)})

In [64]:
cursor.close()
cnx.close()

### Dados na forma de dicionários

Se criar o cursor com o parametro `dictionary=True` ao iterar sobre os resultados estes vêm na forma de dicionários

In [67]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor(dictionary=True)

sql = '''
    SELECT idReading, idSensor, timestamp, value 
    FROM Reading 
    WHERE value BETWEEN %s and %s
'''
data = (5, 50)

cursor.execute(sql, data)

for linha in cursor:
    print(linha)

In [68]:
cursor.close()
cnx.close()

### EXERCÍCIO
Utilize os pacotes `time` e `psutil` para calcular a memoria virtual livre a cada segundo durante 30 segundos, guardando na base de dados. Use
```
...
x = psutil.virtual_memory()
x.free
...
```